## Import Libraries

In [13]:
import os
import json
import cv2
import numpy as np
import base64

## Set Folder Path and File Names

In [14]:
# Set folder path (adjust if needed)
folder_path = '/home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray'

# File names
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
print(f"Found {len(json_files)} JSON files to process.")



Found 1388 JSON files to process.


In [17]:
# List to keep track of saved output images
saved_images = []
saved_text_files = []
saved_fused_images = []

In [18]:
for json_file in json_files:
    try:
        # Construct full paths for the input JSON and the output files
        json_filepath = os.path.join(folder_path, json_file)
        base_name = os.path.splitext(json_file)[0]  # e.g., "slice_0000_subtraction"
        
        # Define output file paths for the annotation image and the class names file
        output_image_filename = os.path.join(folder_path, f"{base_name}_label.png")
        output_text_filename = os.path.join(folder_path, f"{base_name}_label.txt")
        
        print(f"\nProcessing file: {json_file}")
        
        # Load the JSON annotation file
        with open(json_filepath, 'r') as f:
            data = json.load(f)
        
        # Determine image dimensions; default to 300x300 if no image info is available
        height, width = 300, 300
        if "imageData" in data and data["imageData"]:
            # Use embedded base64 imageData to determine dimensions
            imageData = data["imageData"]
            img_bytes = base64.b64decode(imageData)
            img_arr = np.frombuffer(img_bytes, np.uint8)
            decoded_img = cv2.imdecode(img_arr, cv2.IMREAD_COLOR)
            if decoded_img is not None:
                height, width = decoded_img.shape[:2]
        elif "imagePath" in data and data["imagePath"]:
            # If imagePath is provided, load the referenced image file to determine dimensions
            image_file = os.path.join(folder_path, data["imagePath"])
            if os.path.exists(image_file):
                loaded_img = cv2.imread(image_file)
                if loaded_img is not None:
                    height, width = loaded_img.shape[:2]
        
        print(f"Image dimensions determined: {width} x {height}")
        
        # Create a blank (black) image with three RGB channels
        blank_image = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw each polygon for shapes labeled "tumor" with red (BGR: (0, 0, 255))
        for shape in data.get('shapes', []):
            if shape.get("label", "").lower() != "tumor":
                continue  # Only process shapes labeled "tumor"
            
            pts = np.array(shape.get('points', []), dtype=np.float32)
            pts = np.round(pts).astype(np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.fillPoly(blank_image, [pts], (0, 0, 255))
        
        print("Annotated polygons have been drawn on the annotation image.")
        
        # Save the annotation image
        cv2.imwrite(output_image_filename, blank_image)
        print(f"Output annotation image saved to: {output_image_filename}")
        saved_images.append(output_image_filename)
        
        # Write class names to a text file ("background" and "tumor")
        with open(output_text_filename, 'w') as txt_file:
            txt_file.write("background\ntumor")
        print(f"Class names saved to: {output_text_filename}")
        saved_text_files.append(output_text_filename)
        
        # ----------------------------------------------
        # Now overlay (fuse) the red annotations onto the post image
        # ----------------------------------------------
        # Define the post image filename.
        # Remove "_subtraction" from the base name to get the post image base.
        post_base = base_name.replace("_subtraction", "")
        post_image_filename = os.path.join(folder_path, f"{post_base}_post.png")
        if not os.path.exists(post_image_filename):
            print(f"Warning: Post image not found for {post_base}, skipping fusion.")
            continue

        # Load the post image
        post_img = cv2.imread(post_image_filename)
        if post_img is None:
            print(f"Warning: Could not load post image for {post_base}, skipping fusion.")
            continue
        
        # Ensure both images are the same size; if not, resize the annotation image
        if post_img.shape[:2] != blank_image.shape[:2]:
            blank_image_resized = cv2.resize(blank_image, (post_img.shape[1], post_img.shape[0]))
        else:
            blank_image_resized = blank_image
        
        # Create a mask where red annotations exist.
        # We check for pixels where B==0, G==0, and R==255.
        mask = ((blank_image_resized[:, :, 0] == 0) & 
                (blank_image_resized[:, :, 1] == 0) & 
                (blank_image_resized[:, :, 2] == 255)).astype(np.uint8)
        
        # Instead of directly replacing pixels, blend the red annotation with 50% transparency.
        fused_img = post_img.copy()
        # Get indices where the mask is true:
        indices = np.where(mask == 1)
        # For these indices, blend post image pixel with red (annotation) pixel.
        # Blending: output = 0.5 * post + 0.5 * annotation
        fused_img[indices] = (0.5 * post_img[indices] + 0.3 * blank_image_resized[indices]).astype(np.uint8)
        
        # Save the fused image with the naming convention "<post_base>_post_label.png"
        fused_output_filename = os.path.join(folder_path, f"{post_base}_post_label.png")
        cv2.imwrite(fused_output_filename, fused_img)
        print(f"Fused image saved to: {fused_output_filename}")
        saved_fused_images.append(fused_output_filename)
        
    except Exception as e:
        print(f"Error processing {json_file}: {e}")


Processing file: slice_0413_subtraction.json
Image dimensions determined: 300 x 300
Annotated polygons have been drawn on the annotation image.
Output annotation image saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0413_subtraction_label.png
Class names saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0413_subtraction_label.txt
Fused image saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0413_post_label.png

Processing file: slice_0766_subtraction.json
Image dimensions determined: 300 x 300
Annotated polygons have been drawn on the annotation image.
Output annotation image saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0766_subtraction_label.png
Class names saved to: /home/suhkth/Desktop/DL-Breast-MRI-Tumor-Segmentation/convertData2PNG/labeled_mri_gray/slice_0766_subtr

In [71]:
# Report the total counts
print(f"\nTotal number of annotation images saved: {len(saved_images)}")
print(f"Total number of fused images saved: {len(saved_fused_images)}")


Total number of annotation images saved: 1388
Total number of fused images saved: 0
